In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
print(tf.__version__)

1.12.0


In [4]:
tf.enable_eager_execution()

In [5]:
from lstm import LSTM
import numpy as np

batch_size = 64
input_size = 1
hidden_size = 10
cell_size = 10

with tf.device("gpu:0"):
    net = LSTM()
    output_net = tf.keras.layers.Dense(2)

In [6]:
# x = tf.constant(np.zeros((batch_size, input_size)), dtype=tf.float32)

# nh, nc = net([x, h, c])

In [7]:
def generate_xor_batch(batch_size=32):
    x_batch = np.random.randint(2, size=(batch_size, 50, 1))
    
    def xor(sequence):
        result = []
        accum = 0
        for v in sequence:
            accum = accum ^ v
            result.append(accum)
        return result
        
    y_batch = np.array(list(xor(x) for x in x_batch))
    return x_batch, y_batch

def generate_xor_last_batch(batch_size=32):
    x_batch = np.random.randint(2, size=(batch_size, 50, 1))
    
    def xor(sequence):
        result = []
        accum = 0
        for v in sequence:
            accum = accum ^ v
            result.append(accum)
        return result
        
    y_batch = np.array(list(xor(x) for x in x_batch))
    return x_batch, y_batch

In [8]:
def run_rnn(net, xs): 
    h = tf.constant(np.zeros((batch_size, hidden_size)), dtype=tf.float32)
    c = tf.constant(np.zeros((batch_size, cell_size)), dtype=tf.float32)
    
    ys = []
    for x in xs:
        h, c = net([x, h, c])
        ys.append(output_net(h))
        
    return ys

In [ ]:
optimizer = tf.train.AdamOptimizer()

for iteration in range(1000000):
    x_batch, y_batch = generate_xor_batch(batch_size=batch_size)
    x_batch_time = np.swapaxes(x_batch, 0, 1)
    y_batch_time = np.swapaxes(y_batch, 0, 1)
    
    def compute_loss():
        y_batch_time_pred = run_rnn(net, x_batch_time)
        loss = tf.losses.sparse_softmax_cross_entropy(y_batch_time, y_batch_time_pred)
        return loss
    
    with tf.GradientTape() as tape:
        loss_value = compute_loss()
        
    if iteration % 100 == 0:
        print("It: {}, Loss: {}".format(iteration, loss_value))
    
    variables = net.trainable_variables + output_net.trainable_variables
    grads = tape.gradient(loss_value, variables)
    optimizer.apply_gradients(zip(grads, variables))

It: 0, Loss: 0.6931909322738647
It: 100, Loss: 0.6930636763572693
It: 200, Loss: 0.6928489804267883
It: 300, Loss: 0.6922973394393921
It: 400, Loss: 0.6933772563934326
It: 500, Loss: 0.6928486824035645
It: 600, Loss: 0.691215991973877
It: 700, Loss: 0.690264105796814
It: 800, Loss: 0.6842941045761108
It: 900, Loss: 0.6811379790306091
It: 1000, Loss: 0.6769529581069946
It: 1100, Loss: 0.6707047820091248
It: 1200, Loss: 0.668885350227356
It: 1300, Loss: 0.6497145295143127
It: 1400, Loss: 0.20364966988563538
It: 1500, Loss: 0.11225822567939758
It: 1600, Loss: 0.06564738601446152
It: 1700, Loss: 0.04092329367995262
It: 1800, Loss: 0.02956956811249256
It: 1900, Loss: 0.022227099165320396
It: 2000, Loss: 0.01788059063255787
It: 2100, Loss: 0.01480279304087162
It: 2200, Loss: 0.012501670978963375
It: 2300, Loss: 0.00997020024806261
It: 2400, Loss: 0.008596899919211864
It: 2500, Loss: 0.00825677439570427
It: 2600, Loss: 0.007583946455270052
It: 2700, Loss: 0.006588848773390055
It: 2800, Loss: 

In [11]:
tf.test.is_gpu_available()

False